In [1]:
#Importing necassary libraries
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.cluster import SpectralClustering
import random
import matplotlib.pyplot as plt


In [2]:
class WordGraph:   
    """
        Manages the list of words and computes similarities between them to aid in solving the Wordle game.

        Attributes:
            words_full (list): The complete list of words available for guessing.
            words (list): The current list of words, updated as guesses are made.
            k (int): The number of clusters to be used in spectral clustering.
            n (int): The number of words currently available for guessing.
            G (np.array): The similarity graph where each entry G[i][j] represents the similarity between words i and j.
            G_csr (csr_matrix): The Compressed Sparse Row (CSR) representation of the similarity graph for efficient computation.
            original_indices (list): List of original indices of the words in the full word list.
    """
    def __init__(self, word_list, k=5):
        
        """
        Initializes the WordGraph with a list of words and the desired number of clusters.

        Parameters:
            word_list (list): The list of words to be used in the Wordle game.
            k (int, optional): The number of clusters for spectral clustering. Default is 5.
        """
        self.words_full = [word.lower() for word in word_list]
        self.words = self.words_full.copy()
        self.k = k
        self.n = len(self.words)
        self.G = None
        self.G_csr = None
        self.original_indices = list(range(len(self.words)))
    
    def calculate_word_similarity(self, a, b):
        """
        Calculates the similarity score between two words based on the number of matching letters and shared characters.

        Parameters:
            a (str): The first word.
            b (str): The second word.

        Returns:
            float: The similarity score between word 'a' and word 'b'.
        """
        assert len(a) == len(b), f'{a} is a different length than {b}'
        score = sum(1 for i in range(len(a)) if a[i] == b[i])
        return (score + len(set(a).intersection(b))) / 10


    def make_similarity_graph(self):
        """
        Constructs the similarity graph for the current list of words. This graph is used for spectral clustering.
        The graph is only constructed if it hasn't been already, to avoid redundant computations.
        """
        if self.G is not None:
            return # Skip if the graph is already computed
        
        n = len(self.words)
        self.G = np.zeros((n, n))
        for i in range(n):
            for j in range(i + 1, n):
                self.G[i, j] = self.calculate_word_similarity(self.words[i], self.words[j])
                self.G[j, i] = self.G[i, j]
        self.G_csr = csr_matrix(self.G)

    def filter_words(self, guess, feedback):
        """
        Filters the list of words based on the feedback received from a guess in the Wordle game.

        Parameters:
            guess (str): The word that was guessed.
            feedback (list): The feedback received for the guess, indicating correct and incorrect letters.

        Returns:
            bool: True if there are words left after filtering, False otherwise.
        """
        self.words = [word for word in self.words if self._is_word_valid(word, guess, feedback)]
        self.n = len(self.words)
        return bool(self.words)
    
    def _is_word_valid(self, word, guess, feedback):
        """
        Determines if a word is still valid based on the feedback from a guess.

        Parameters:
            word (str): The word to check.
            guess (str): The guessed word.
            feedback (list): The feedback received for the guess.

        Returns:
            bool: True if the word is still valid, False otherwise.
        """
        allowed_chars = set(guess[i] for i, f in enumerate(feedback) if f > 0)
        for i, f in enumerate(feedback):
            if f == 2:
                if guess[i] != word[i]:
                    return False
            elif f == 1:
                if guess[i] == word[i] or guess[i] not in word:
                    return False
            elif f == 0:
                if guess[i] in word and guess[i] not in allowed_chars:
                    return False
        return True

    def reset(self):
        """
        Resets the WordGraph to its initial state for a new game or training session.
        This includes resetting the list of words and the similarity graph.
        """
        self.words = self.words_full.copy()
        self.n = len(self.words)
        self.G = None  

In [3]:
class SpectralClusterer:
    """
        Performs spectral clustering on the words to group similar words together. This clustering aids the AI in making strategic guesses by focusing on specific groups of words.

        Attributes:
            word_graph (WordGraph): An instance of the WordGraph class, providing the necessary word data and similarity graph.
            clusters (np.array): Stores the cluster assignments for each word.
    """
    def __init__(self, word_graph):
        """
        Initializes the SpectralClusterer with a WordGraph instance.

        Parameters:
            word_graph (WordGraph): The WordGraph instance containing the word list and similarity data.
        """
        self.word_graph = word_graph
        self.clusters = None

    def make_clusters(self):
        """
        Performs spectral clustering on the words based on the similarity graph from the WordGraph. Assigns words to clusters for the AI to use during guessing.

        This method handles the case where the number of words is less than or equal to the number of desired clusters by assigning each word to its own cluster.
        """
        if len(self.word_graph.words) > self.word_graph.k:
            # Create the spectral clustering based on the similarity graph
            self.clusters = SpectralClustering(
                n_clusters=self.word_graph.k,
                assign_labels='discretize',
                affinity='precomputed',
                random_state=13
            ).fit_predict(self.word_graph.G_csr)
        else:
            # If the number of words is less or equal to the number of clusters, assign each word to its own cluster
            self.clusters = np.arange(len(self.word_graph.words))

    def get_cluster_state(self):
        """
        Calculates the current state of the clusters, which is used to inform the AI's decision-making.

        The state is represented by the proportion of words in each cluster, converted into a string format for compatibility with the Q-learning model.

        Returns:
            str: A string representation of the current state based on the cluster proportions.
        """
        self.word_graph.make_similarity_graph() 
        # Calculate the proportion of words in each cluster
        cluster_counts = np.bincount(self.clusters, minlength=self.word_graph.k)
        total_words = len(self.word_graph.words)
        cluster_proportions = cluster_counts / total_words

        # Convert proportions to a string state
        state = ''.join(str(round(proportion, 2)) for proportion in cluster_proportions)
        return state

    def get_cluster_centroid(self, cluster_index):
        """
        Identifies the centroid of a given cluster. The centroid is the word in the cluster with the highest aggregate similarity to other words in the same cluster.

        Parameters:
            cluster_index (int): The index of the cluster.

        Returns:
            str: The centroid word of the specified cluster, or None if the cluster is empty.
        """
        # Find the indices of words in the specified cluster.
        cluster_indices = np.where(self.clusters == cluster_index)[0]
        
        # Map these indices to the current list of words.
        valid_indices = [i for i in cluster_indices if i < len(self.word_graph.words)]
        
        # If there are no valid words in the cluster, return None.
        if not valid_indices:
            print(f"No words in cluster {cluster_index}, choosing from another cluster.")
            valid_indices = [i for i in range(len(self.word_graph.words)) 
                             if i not in self.clusters or self.clusters[i] != cluster_index]
            if not valid_indices:
                print("No valid words remaining in any cluster.")
                return None

        # Now, valid_indices contains only indices of the current words, post-filtering.
        # Calculate the word scores based on similarities within the cluster.
        word_scores = np.sum(self.word_graph.G[valid_indices, :][:, valid_indices], axis=1)
        max_score_index = np.argmax(word_scores)

        # The word corresponding to the max_score_index is the centroid.
        centroid_word = self.word_graph.words[valid_indices[max_score_index]]
        return centroid_word
    
    def update_clusters_after_filtering(self):
        """
        Updates the cluster assignments after words have been filtered out based on the game's feedback. This ensures that the clusters remain consistent with the current list of words.

        The method retains cluster assignments only for words that are still valid after filtering.
        """
        valid_indices = set(self.word_graph.original_indices)
        self.clusters = np.array([index for index in self.clusters if index in valid_indices])

In [4]:
class Grader:
    """
        Simulates the feedback mechanism of the Wordle game. It generates feedback for guesses based on a chosen secret word.

        Attributes:
            words (list): A list of possible words that can be chosen as the secret word for the game.
            word (str): The current secret word selected for the game.
    """
    def __init__(self, words):
        """
        Initializes the Grader with a list of possible words.

        Parameters:
            words (list): A list of words to be used in the Wordle game.
        """
        self.words = words
        self.word = None

    def pick_word(self):
        """
        Randomly selects a word from the list of words to be used as the secret word in the Wordle game.
        """
        self.word = np.random.choice(self.words)

    def evaluate_guess(self, guess):
        """
        Evaluates the given guess against the secret word and generates feedback. The feedback indicates which letters are correct and in the correct position (2), correct but in the wrong position (1), or not in the word (0).

        Parameters:
            guess (str): The guessed word.

        Returns:
            np.array: An array of feedback corresponding to each letter in the guess.
        """
        assert len(guess) == 5, "Guess must be a 5-letter word"
        feedback = np.zeros(5, dtype=int)
        word_copy = list(self.word)

        # First pass for correct letters in correct places
        for i in range(5):
            if guess[i] == self.word[i]:
                feedback[i] = 2
                word_copy[i] = None  # Mark this letter as evaluated

        # Second pass for correct letters in wrong places
        for i in range(5):
            if feedback[i] == 0 and guess[i] in word_copy:
                feedback[i] = 1
                word_copy[word_copy.index(guess[i])] = None  # Mark this letter as evaluated
        return feedback


In [5]:
class QLearner:
    """
        Implements the Q-learning algorithm for the Wordle solver. Manages and updates the Q-table based on the game's feedback.

        Attributes:
            cluster_count (int): The number of word clusters used for decision making.
            alpha (float): The learning rate, determining how new information affects existing knowledge in the Q-table.
            gamma (float): The discount factor, valuing future rewards in the Q-learning updates.
            random_action_rate (float): The rate of taking random actions to explore different strategies.
            random_action_decay_rate (float): The decay rate of the random action rate over time.
            current_state (str): The current state of the game.
            current_action (int): The current action taken by the AI.
            qtable (dict): The Q-table storing the values for state-action pairs.
    """
    def __init__(self, cluster_count, alpha, gamma, random_action_rate, random_action_decay_rate):
        """
        Initializes the QLearner with specified parameters for the Q-learning algorithm.

        Parameters:
            cluster_count (int): The number of clusters to consider in decision-making.
            alpha (float): The learning rate for Q-learning.
            gamma (float): The discount factor for future rewards.
            random_action_rate (float): The initial rate of taking random actions.
            random_action_decay_rate (float): The decay rate for the random action rate.
        """
        self.cluster_count = cluster_count
        self.alpha = alpha
        self.gamma = gamma
        self.random_action_rate = random_action_rate
        self.random_action_decay_rate = random_action_decay_rate
        self.current_state = None
        self.current_action = None
        self.qtable = {}

    def initialize_state(self, state):
        """
        Initializes the current state in the Q-table if it hasn't been encountered before.

        Parameters:
            state (str): The current state to be initialized.
        """
        self.current_state = state
        if state not in self.qtable:
            self.qtable[state] = np.zeros(self.cluster_count)

    def get_random_action(self):
        """
        Selects a random action from the available cluster choices.

        Returns:
            int: A randomly selected action (cluster index).
        """
        return random.randint(0, self.cluster_count - 1)

    def update_qtable(self, next_state, reward):
        """
        Updates the Q-table based on the reward received and the next state. It uses the Q-learning formula to adjust the value of the current state-action pair.

        Parameters:
            next_state (str): The state that will follow after the current action.
            reward (float): The reward received after taking the current action.

        Returns:
            int: The next action to take based on the updated Q-table.
        """
        if next_state not in self.qtable:
            self.qtable[next_state] = np.zeros(self.cluster_count)

        next_action = np.argmax(self.qtable[next_state])
        self.qtable[self.current_state][self.current_action] *= (1 - self.alpha)
        self.qtable[self.current_state][self.current_action] += self.alpha * (reward + self.gamma * self.qtable[next_state][next_action])

        return next_action


    def get_action(self, next_state, reward):
        """
        Determines the next action to take. It either chooses a random action or the best action based on the Q-table, depending on the current random action rate.

        Parameters:
            next_state (str): The next state of the game.
            reward (float): The reward received from the previous action.

        Returns:
            int: The chosen action for the next state.
        """
        if reward is None:
            reward = 0
        next_action = self.update_qtable(next_state, reward)
        self.random_action_rate *= self.random_action_decay_rate
        if random.random() <= self.random_action_rate:
            action = self.get_random_action()
        else:
            action = next_action

        self.current_state = next_state
        self.current_action = action
        return action

    def reset_qtable(self):
        """
        Reset the Q-table to an empty state for a new training session.
        """
        self.qtable = {}


In [6]:
class WordleBot:
    """
    The main class that integrates all components to play the Wordle game. It manages the game flow, making guesses based on learned strategies, and updating its learning model.

    Attributes:
        word_graph (WordGraph): An instance of the WordGraph class for managing words and their similarities.
        clusterer (SpectralClusterer): An instance of the SpectralClusterer class for clustering words.
        learner (QLearner): An instance of the QLearner class for making decisions based on Q-learning.
        grader (Grader): An instance of the Grader class for evaluating guesses and providing feedback.
    """
    def __init__(self, word_list, cluster_count, alpha, gamma, random_action_rate, random_action_decay_rate):
        """
        Initializes the WordleBot with the necessary components and parameters for the Q-learning algorithm.

        Parameters:
            word_list (list): The list of words to be used in the game.
            cluster_count (int): The number of clusters for spectral clustering.
            alpha (float): The learning rate for Q-learning.
            gamma (float): The discount factor for Q-learning.
            random_action_rate (float): The initial rate of taking random actions.
            random_action_decay_rate (float): The decay rate for the random action rate.
        """
        self.word_graph = WordGraph(word_list)
        self.word_graph.make_similarity_graph()
        self.clusterer = SpectralClusterer(self.word_graph)
        self.learner = QLearner(cluster_count, alpha, gamma, random_action_rate, random_action_decay_rate)
        self.grader = Grader(word_list)



    def play_game(self, training=True):
        """
        Plays a single game of Wordle, making guesses and learning from the feedback.

        Parameters:
            training (bool): Indicates whether the game is being played for training. If False, the Q-table is not updated.

        Returns:
            int: The number of attempts taken to guess the word, or None if the word was not guessed within the allowed attempts.
        """
        self.grader.pick_word()
        secret_word = self.grader.word 
        print(f"Secret Word: {secret_word}")

        reward = 0  # Initialize reward

        for attempt in range(1, 7):  # Wordle allows up to 6 attempts
            print(f"\nAttempt {attempt}:")
            
            # Make clusters and choose action (guess) based on Q-learning
            self.clusterer.make_clusters()
            state = self.clusterer.get_cluster_state()
            self.learner.initialize_state(state)

            # For the first attempt, choose a random action
            if attempt == 1:
                action = self.learner.get_random_action()
            else:
                action = self.learner.get_action(state, reward)

            guess = self.clusterer.get_cluster_centroid(action)
            print(f"Guessing word: {guess}")

            # Evaluate the guess and update

            if guess is None:
                print("No valid guess available. Skipping attempt.")
                continue

            feedback = self.grader.evaluate_guess(guess)

            if not self.word_graph.filter_words(guess, feedback):
                print("No valid words remaining. Skipping further attempts.")
                break

            # Check for win condition
            if all(f == 2 for f in feedback):
                print("Word guessed correctly!")
                return attempt

            # Update the word list and Q-table based on feedback
            self.word_graph.filter_words(guess, feedback)
            self.clusterer.update_clusters_after_filtering()
            new_state = self.clusterer.get_cluster_state()

            reward = self.calculate_reward(feedback)

            # Update Q-table only if in training mode
            if training:
                self.learner.get_action(new_state, reward)
            else:
                # Inference mode: Use Q-table but do not update it
                action = np.argmax(self.learner.qtable[state])
                print(f"Chosen action based on Q-table (inference mode): {action}")
        return attempt

    
        #print("Failed to guess the word after 6 attempts.")
        #return None  # Indicate that the bot failed to guess within 6 attempts
        
    def reset(self):
        """
        Resets the WordleBot for a new game, including resetting the WordGraph.
        """
        self.word_graph.reset()

    def calculate_reward(self, feedback):
        """
        Calculates the reward based on the feedback received for a guess. The reward is a sum of the feedback values, with a bonus for guessing the word correctly.

        Parameters:
            feedback (list): The feedback received for the guess.

        Returns:
            int: The calculated reward.
        """
        reward = sum(feedback) + (1000 if sum(feedback) == 10 else 0)
        print(f"Calculated reward: {reward}")
        return reward

In [7]:
class Trainer:
    """
    Manages the training process for the WordleBot. This class is responsible for running multiple games to train the AI, collecting results, and assessing the performance.

    Attributes:
        wordle_bot (WordleBot): An instance of the WordleBot class, which will be trained.
        num_games (int): The number of games to play during the training session.
        game_results (list): A list to store the results of each game (number of attempts taken to guess the word).
    """
    def __init__(self, wordle_bot, num_games):
        """
        Initializes the Trainer with a WordleBot instance and the number of games to be played for training.

        Parameters:
            wordle_bot (WordleBot): The instance of WordleBot to be trained.
            num_games (int): The total number of games to play for the training.
        """
        self.wordle_bot = wordle_bot
        self.num_games = num_games
        self.game_results = []

    def train(self):
        """
        Runs the training process by playing the specified number of games with the WordleBot. The performance of the WordleBot in each game is recorded.

        This method also handles resetting the WordleBot after each game to ensure each game starts with a fresh state.
        """
        print("Starting training...")
        for _ in range(self.num_games):
            attempts = self.wordle_bot.play_game()
            self.game_results.append(attempts)
            print(f"Game {_+1}: Word guessed in {attempts} attempts.")
            self.wordle_bot.reset()
        print("Training complete.")
        
    def get_results(self):
        """
        Retrieves the results of the training session. This includes the number of attempts taken by the WordleBot to guess the word in each game.

        Returns:
            list: A list containing the results (number of attempts) of each game played during the training session.
        """
        return self.game_results


In [8]:
def run_training_with_parameters(word_list, num_games, parameter_sets):
    """
    Runs training sessions for the WordleBot with various parameter configurations.

    Parameters:
        word_list (list): The list of words to be used in the game.
        num_games (int): The number of games to play in each training session.
        parameter_sets (list): A list of dictionaries, each representing a set of parameters for the WordleBot.

    Returns:
        list: A list of tuples, each containing the parameters used and the corresponding game results.
    """
    performance_results = []

    for params in parameter_sets:
        # Initialize WordleBot with a set of parameters
        wordle_bot = WordleBot(word_list, **params)
        
        # Train the WordleBot and collect game results
        trainer = Trainer(wordle_bot, num_games)
        trainer.train()
        performance_results.append((params, trainer.get_results()))

    return performance_results

def plot_performance(performance_results):
    """
    Plots the performance of the WordleBot for each set of parameters over the training games.

    Parameters:
        performance_results (list): The results from the training sessions, including parameters and game results.
    """
    # Create subplots for each set of parameters
    fig, axes = plt.subplots(len(performance_results), 1, figsize=(10, 5 * len(performance_results)))

    # Handle the case of a single set of parameters
    if len(performance_results) == 1:
        axes = [axes]

    # Plot the results for each set of parameters
    for ax, (params, results) in zip(axes, performance_results):
        ax.plot(results, label=f"Params: {params}")
        ax.set_xlabel('Game Number')
        ax.set_ylabel('Attempts Taken')
        ax.set_title(f"Performance with Parameters: {params}")
        ax.legend()

    plt.tight_layout()
    plt.show()


def report_training_performance(performance_results):
    """
    Reports the training performance for each set of parameters, including average attempts, best and worst performance.

    Parameters:
        performance_results (list): The results from the training sessions, including parameters and game results.
    """
    for params, results in performance_results:
        # Calculate key performance metrics
        avg_attempts = np.mean(results)
        best_performance = min(results)
        worst_performance = max(results)

        # Print the performance report
        print(f"Parameters: {params}")
        print(f"Average Attempts: {avg_attempts}, Best: {best_performance}, Worst: {worst_performance}\n")


In [9]:
#2300 words pool:
word_list=["cigar","rebut","sissy","humph","awake","blush","focal","evade","naval","serve","heath","dwarf","model","karma","stink","grade","quiet","bench","abate","feign","major","death","fresh","crust","stool","colon","abase","marry","react","batty","pride","floss","helix","croak","staff","paper","unfed","whelp","trawl","outdo","adobe","crazy","sower","repay","digit","crate","cluck","spike","mimic","pound","maxim","linen","unmet","flesh","booby","forth","first","stand","belly","ivory","seedy","print","yearn","drain","bribe","stout","panel","crass","flume","offal","agree","error","swirl","argue","bleed","delta","flick","totem","wooer","front","shrub","parry","biome","lapel","start","greet","goner","golem","lusty","loopy","round","audit","lying","gamma","labor","islet","civic","forge","corny","moult","basic","salad","agate","spicy","spray","essay","fjord","spend","kebab","guild","aback","motor","alone","hatch","hyper","thumb","dowry","ought","belch","dutch","pilot","tweed","comet","jaunt","enema","steed","abyss","growl","fling","dozen","boozy","erode","world","gouge","click","briar","great","altar","pulpy","blurt","coast","duchy","groin","fixer","group","rogue","badly","smart","pithy","gaudy","chill","heron","vodka","finer","surer","radio","rouge","perch","retch","wrote","clock","tilde","store","prove","bring","solve","cheat","grime","exult","usher","epoch","triad","break","rhino","viral","conic","masse","sonic","vital","trace","using","peach","champ","baton","brake","pluck","craze","gripe","weary","picky","acute","ferry","aside","tapir","troll","unify","rebus","boost","truss","siege","tiger","banal","slump","crank","gorge","query","drink","favor","abbey","tangy","panic","solar","shire","proxy","point","robot","prick","wince","crimp","knoll","sugar","whack","mount","perky","could","wrung","light","those","moist","shard","pleat","aloft","skill","elder","frame","humor","pause","ulcer","ultra","robin","cynic","agora","aroma","caulk","shake","pupal","dodge","swill","tacit","other","thorn","trove","bloke","vivid","spill","chant","choke","rupee","nasty","mourn","ahead","brine","cloth","hoard","sweet","month","lapse","watch","today","focus","smelt","tease","cater","movie","lynch","saute","allow","renew","their","slosh","purge","chest","depot","epoxy","nymph","found","shall","harry","stove","lowly","snout","trope","fewer","shawl","natal","fibre","comma","foray","scare","stair","black","squad","royal","chunk","mince","slave","shame","cheek","ample","flair","foyer","cargo","oxide","plant","olive","inert","askew","heist","shown","zesty","hasty","trash","fella","larva","forgo","story","hairy","train","homer","badge","midst","canny","fetus","butch","farce","slung","tipsy","metal","yield","delve","being","scour","glass","gamer","scrap","money","hinge","album","vouch","asset","tiara","crept","bayou","atoll","manor","creak","showy","phase","froth","depth","gloom","flood","trait","girth","piety","payer","goose","float","donor","atone","primo","apron","blown","cacao","loser","input","gloat","awful","brink","smite","beady","rusty","retro","droll","gawky","hutch","pinto","gaily","egret","lilac","sever","field","fluff","hydro","flack","agape","wench","voice","stead","stalk","berth","madam","night","bland","liver","wedge","augur","roomy","wacky","flock","angry","bobby","trite","aphid","tryst","midge","power","elope","cinch","motto","stomp","upset","bluff","cramp","quart","coyly","youth","rhyme","buggy","alien","smear","unfit","patty","cling","glean","label","hunky","khaki","poker","gruel","twice","twang","shrug","treat","unlit","waste","merit","woven","octal","needy","clown","widow","irony","ruder","gauze","chief","onset","prize","fungi","charm","gully","inter","whoop","taunt","leery","class","theme","lofty","tibia","booze","alpha","thyme","eclat","doubt","parer","chute","stick","trice","alike","sooth","recap","saint","liege","glory","grate","admit","brisk","soggy","usurp","scald","scorn","leave","twine","sting","bough","marsh","sloth","dandy","vigor","howdy","enjoy","valid","ionic","equal","unset","floor","catch","spade","stein","exist","quirk","denim","grove","spiel","mummy","fault","foggy","flout","carry","sneak","libel","waltz","aptly","piney","inept","aloud","photo","dream","stale","vomit","ombre","fanny","unite","snarl","baker","there","glyph","pooch","hippy","spell","folly","louse","gulch","vault","godly","threw","fleet","grave","inane","shock","crave","spite","valve","skimp","claim","rainy","musty","pique","daddy","quasi","arise","aging","valet","opium","avert","stuck","recut","mulch","genre","plume","rifle","count","incur","total","wrest","mocha","deter","study","lover","safer","rivet","funny","smoke","mound","undue","sedan","pagan","swine","guile","gusty","equip","tough","canoe","chaos","covet","human","udder","lunch","blast","stray","manga","melee","lefty","quick","paste","given","octet","risen","groan","leaky","grind","carve","loose","sadly","spilt","apple","slack","honey","final","sheen","eerie","minty","slick","derby","wharf","spelt","coach","erupt","singe","price","spawn","fairy","jiffy","filmy","stack","chose","sleep","ardor","nanny","niece","woozy","handy","grace","ditto","stank","cream","usual","diode","valor","angle","ninja","muddy","chase","reply","prone","spoil","heart","shade","diner","arson","onion","sleet","dowel","couch","palsy","bowel","smile","evoke","creek","lance","eagle","idiot","siren","built","embed","award","dross","annul","goody","frown","patio","laden","humid","elite","lymph","edify","might","reset","visit","gusto","purse","vapor","crock","write","sunny","loath","chaff","slide","queer","venom","stamp","sorry","still","acorn","aping","pushy","tamer","hater","mania","awoke","brawn","swift","exile","birch","lucky","freer","risky","ghost","plier","lunar","winch","snare","nurse","house","borax","nicer","lurch","exalt","about","savvy","toxin","tunic","pried","inlay","chump","lanky","cress","eater","elude","cycle","kitty","boule","moron","tenet","place","lobby","plush","vigil","index","blink","clung","qualm","croup","clink","juicy","stage","decay","nerve","flier","shaft","crook","clean","china","ridge","vowel","gnome","snuck","icing","spiny","rigor","snail","flown","rabid","prose","thank","poppy","budge","fiber","moldy","dowdy","kneel","track","caddy","quell","dumpy","paler","swore","rebar","scuba","splat","flyer","horny","mason","doing","ozone","amply","molar","ovary","beset","queue","cliff","magic","truce","sport","fritz","edict","twirl","verse","llama","eaten","range","whisk","hovel","rehab","macaw","sigma","spout","verve","sushi","dying","fetid","brain","buddy","thump","scion","candy","chord","basin","march","crowd","arbor","gayly","musky","stain","dally","bless","bravo","stung","title","ruler","kiosk","blond","ennui","layer","fluid","tatty","score","cutie","zebra","barge","matey","bluer","aider","shook","river","privy","betel","frisk","bongo","begun","azure","weave","genie","sound","glove","braid","scope","wryly","rover","assay","ocean","bloom","irate","later","woken","silky","wreck","dwelt","slate","smack","solid","amaze","hazel","wrist","jolly","globe","flint","rouse","civil","vista","relax","cover","alive","beech","jetty","bliss","vocal","often","dolly","eight","joker","since","event","ensue","shunt","diver","poser","worst","sweep","alley","creed","anime","leafy","bosom","dunce","stare","pudgy","waive","choir","stood","spoke","outgo","delay","bilge","ideal","clasp","seize","hotly","laugh","sieve","block","meant","grape","noose","hardy","shied","drawl","daisy","putty","strut","burnt","tulip","crick","idyll","vixen","furor","geeky","cough","naive","shoal","stork","bathe","aunty","check","prime","brass","outer","furry","razor","elect","evict","imply","demur","quota","haven","cavil","swear","crump","dough","gavel","wagon","salon","nudge","harem","pitch","sworn","pupil","excel","stony","cabin","unzip","queen","trout","polyp","earth","storm","until","taper","enter","child","adopt","minor","fatty","husky","brave","filet","slime","glint","tread","steal","regal","guest","every","murky","share","spore","hoist","buxom","inner","otter","dimly","level","sumac","donut","stilt","arena","sheet","scrub","fancy","slimy","pearl","silly","porch","dingo","sepia","amble","shady","bread","friar","reign","dairy","quill","cross","brood","tuber","shear","posit","blank","villa","shank","piggy","freak","which","among","fecal","shell","would","algae","large","rabbi","agony","amuse","bushy","copse","swoon","knife","pouch","ascot","plane","crown","urban","snide","relay","abide","viola","rajah","straw","dilly","crash","amass","third","trick","tutor","woody","blurb","grief","disco","where","sassy","beach","sauna","comic","clued","creep","caste","graze","snuff","frock","gonad","drunk","prong","lurid","steel","halve","buyer","vinyl","utile","smell","adage","worry","tasty","local","trade","finch","ashen","modal","gaunt","clove","enact","adorn","roast","speck","sheik","missy","grunt","snoop","party","touch","mafia","emcee","array","south","vapid","jelly","skulk","angst","tubal","lower","crest","sweat","cyber","adore","tardy","swami","notch","groom","roach","hitch","young","align","ready","frond","strap","puree","realm","venue","swarm","offer","seven","dryer","diary","dryly","drank","acrid","heady","theta","junto","pixie","quoth","bonus","shalt","penne","amend","datum","build","piano","shelf","lodge","suing","rearm","coral","ramen","worth","psalm","infer","overt","mayor","ovoid","glide","usage","poise","randy","chuck","prank","fishy","tooth","ether","drove","idler","swath","stint","while","begat","apply","slang","tarot","radar","credo","aware","canon","shift","timer","bylaw","serum","three","steak","iliac","shirk","blunt","puppy","penal","joist","bunny","shape","beget","wheel","adept","stunt","stole","topaz","chore","fluke","afoot","bloat","bully","dense","caper","sneer","boxer","jumbo","lunge","space","avail","short","slurp","loyal","flirt","pizza","conch","tempo","droop","plate","bible","plunk","afoul","savoy","steep","agile","stake","dwell","knave","beard","arose","motif","smash","broil","glare","shove","baggy","mammy","swamp","along","rugby","wager","quack","squat","snaky","debit","mange","skate","ninth","joust","tramp","spurn","medal","micro","rebel","flank","learn","nadir","maple","comfy","remit","gruff","ester","least","mogul","fetch","cause","oaken","aglow","meaty","gaffe","shyly","racer","prowl","thief","stern","poesy","rocky","tweet","waist","spire","grope","havoc","patsy","truly","forty","deity","uncle","swish","giver","preen","bevel","lemur","draft","slope","annoy","lingo","bleak","ditty","curly","cedar","dirge","grown","horde","drool","shuck","crypt","cumin","stock","gravy","locus","wider","breed","quite","chafe","cache","blimp","deign","fiend","logic","cheap","elide","rigid","false","renal","pence","rowdy","shoot","blaze","envoy","posse","brief","never","abort","mouse","mucky","sulky","fiery","media","trunk","yeast","clear","skunk","scalp","bitty","cider","koala","duvet","segue","creme","super","grill","after","owner","ember","reach","nobly","empty","speed","gipsy","recur","smock","dread","merge","burst","kappa","amity","shaky","hover","carol","snort","synod","faint","haunt","flour","chair","detox","shrew","tense","plied","quark","burly","novel","waxen","stoic","jerky","blitz","beefy","lyric","hussy","towel","quilt","below","bingo","wispy","brash","scone","toast","easel","saucy","value","spice","honor","route","sharp","bawdy","radii","skull","phony","issue","lager","swell","urine","gassy","trial","flora","upper","latch","wight","brick","retry","holly","decal","grass","shack","dogma","mover","defer","sober","optic","crier","vying","nomad","flute","hippo","shark","drier","obese","bugle","tawny","chalk","feast","ruddy","pedal","scarf","cruel","bleat","tidal","slush","semen","windy","dusty","sally","igloo","nerdy","jewel","shone","whale","hymen","abuse","fugue","elbow","crumb","pansy","welsh","syrup","terse","suave","gamut","swung","drake","freed","afire","shirt","grout","oddly","tithe","plaid","dummy","broom","blind","torch","enemy","again","tying","pesky","alter","gazer","noble","ethos","bride","extol","decor","hobby","beast","idiom","utter","these","sixth","alarm","erase","elegy","spunk","piper","scaly","scold","hefty","chick","sooty","canal","whiny","slash","quake","joint","swept","prude","heavy","wield","femme","lasso","maize","shale","screw","spree","smoky","whiff","scent","glade","spent","prism","stoke","riper","orbit","cocoa","guilt","humus","shush","table","smirk","wrong","noisy","alert","shiny","elate","resin","whole","hunch","pixel","polar","hotel","sword","cleat","mango","rumba","puffy","filly","billy","leash","clout","dance","ovate","facet","chili","paint","liner","curio","salty","audio","snake","fable","cloak","navel","spurt","pesto","balmy","flash","unwed","early","churn","weedy","stump","lease","witty","wimpy","spoof","saner","blend","salsa","thick","warty","manic","blare","squib","spoon","probe","crepe","knack","force","debut","order","haste","teeth","agent","widen","icily","slice","ingot","clash","juror","blood","abode","throw","unity","pivot","slept","troop","spare","sewer","parse","morph","cacti","tacky","spool","demon","moody","annex","begin","fuzzy","patch","water","lumpy","admin","omega","limit","tabby","macho","aisle","skiff","basis","plank","verge","botch","crawl","lousy","slain","cubic","raise","wrack","guide","foist","cameo","under","actor","revue","fraud","harpy","scoop","climb","refer","olden","clerk","debar","tally","ethic","cairn","tulle","ghoul","hilly","crude","apart","scale","older","plain","sperm","briny","abbot","rerun","quest","crisp","bound","befit","drawn","suite","itchy","cheer","bagel","guess","broad","axiom","chard","caput","leant","harsh","curse","proud","swing","opine","taste","lupus","gumbo","miner","green","chasm","lipid","topic","armor","brush","crane","mural","abled","habit","bossy","maker","dusky","dizzy","lithe","brook","jazzy","fifty","sense","giant","surly","legal","fatal","flunk","began","prune","small","slant","scoff","torus","ninny","covey","viper","taken","moral","vogue","owing","token","entry","booth","voter","chide","elfin","ebony","neigh","minim","melon","kneed","decoy","voila","ankle","arrow","mushy","tribe","cease","eager","birth","graph","odder","terra","weird","tried","clack","color","rough","weigh","uncut","ladle","strip","craft","minus","dicey","titan","lucid","vicar","dress","ditch","gypsy","pasta","taffy","flame","swoop","aloof","sight","broke","teary","chart","sixty","wordy","sheer","leper","nosey","bulge","savor","clamp","funky","foamy","toxic","brand","plumb","dingy","butte","drill","tripe","bicep","tenor","krill","worse","drama","hyena","think","ratio","cobra","basil","scrum","bused","phone","court","camel","proof","heard","angel","petal","pouty","throb","maybe","fetal","sprig","spine","shout","cadet","macro","dodgy","satyr","rarer","binge","trend","nutty","leapt","amiss","split","myrrh","width","sonar","tower","baron","fever","waver","spark","belie","sloop","expel","smote","baler","above","north","wafer","scant","frill","awash","snack","scowl","frail","drift","limbo","fence","motel","ounce","wreak","revel","talon","prior","knelt","cello","flake","debug","anode","crime","salve","scout","imbue","pinky","stave","vague","chock","fight","video","stone","teach","cleft","frost","prawn","booty","twist","apnea","stiff","plaza","ledge","tweak","board","grant","medic","bacon","cable","brawl","slunk","raspy","forum","drone","women","mucus","boast","toddy","coven","tumor","truer","wrath","stall","steam","axial","purer","daily","trail","niche","mealy","juice","nylon","plump","merry","flail","papal","wheat","berry","cower","erect","brute","leggy","snipe","sinew","skier","penny","jumpy","rally","umbra","scary","modem","gross","avian","greed","satin","tonic","parka","sniff","livid","stark","trump","giddy","reuse","taboo","avoid","quote","devil","liken","gloss","gayer","beret","noise","gland","dealt","sling","rumor","opera","thigh","tonga","flare","wound","white","bulky","etude","horse","circa","paddy","inbox","fizzy","grain","exert","surge","gleam","belle","salvo","crush","fruit","sappy","taker","tract","ovine","spiky","frank","reedy","filth","spasm","heave","mambo","right","clank","trust","lumen","borne","spook","sauce","amber","lathe","carat","corer","dirty","slyly","affix","alloy","taint","sheep","kinky","wooly","mauve","flung","yacht","fried","quail","brunt","grimy","curvy","cagey","rinse","deuce","state","grasp","milky","bison","graft","sandy","baste","flask","hedge","girly","swash","boney","coupe","endow","abhor","welch","blade","tight","geese","miser","mirth","cloud","cabal","leech","close","tenth","pecan","droit","grail","clone","guise","ralph","tango","biddy","smith","mower","payee","serif","drape","fifth","spank","glaze","allot","truck","kayak","virus","testy","tepee","fully","zonal","metro","curry","grand","banjo","axion","bezel","occur","chain","nasal","gooey","filer","brace","allay","pubic","raven","plead","gnash","flaky","munch","dully","eking","thing","slink","hurry","theft","shorn","pygmy","ranch","wring","lemon","shore","mamma","froze","newer","style","moose","antic","drown","vegan","chess","guppy","union","lever","lorry","image","cabby","druid","exact","truth","dopey","spear","cried","chime","crony","stunk","timid","batch","gauge","rotor","crack","curve","latte","witch","bunch","repel","anvil","soapy","meter","broth","madly","dried","scene","known","magma","roost","woman","thong","punch","pasty","downy","knead","whirl","rapid","clang","anger","drive","goofy","email","music","stuff","bleep","rider","mecca","folio","setup","verso","quash","fauna","gummy","happy","newly","fussy","relic","guava","ratty","fudge","femur","chirp","forte","alibi","whine","petty","golly","plait","fleck","felon","gourd","brown","thrum","ficus","stash","decry","wiser","junta","visor","daunt","scree","impel","await","press","whose","turbo","stoop","speak","mangy","eying","inlet","crone","pulse","mossy","staid","hence","pinch","teddy","sully","snore","ripen","snowy","attic","going","leach","mouth","hound","clump","tonal","bigot","peril","piece","blame","haute","spied","undid","intro","basal","shine","gecko","rodeo","guard","steer","loamy","scamp","scram","manly","hello","vaunt","organ","feral","knock","extra","condo","adapt","willy","polka","rayon","skirt","faith","torso","match","mercy","tepid","sleek","riser","twixt","peace","flush","catty","login","eject","roger","rival","untie","refit","aorta","adult","judge","rower","artsy","rural","shave"]

#parameters tuning:
parameter_sets = [
#    {'cluster_count': 3, 'alpha': 0.1, 'gamma': 0.5, 'random_action_rate': 0.5, 'random_action_decay_rate': 0.95},
#    {'cluster_count': 4, 'alpha': 0.2, 'gamma': 0.2, 'random_action_rate': 0.5, 'random_action_decay_rate': 0.95},
#    {'cluster_count': 4, 'alpha': 0.6, 'gamma': 0.4, 'random_action_rate': 0.5, 'random_action_decay_rate': 0.95},
    {'cluster_count': 5, 'alpha': 0.3, 'gamma': 0.8, 'random_action_rate': 0.5, 'random_action_decay_rate': 0.95}
    ]

#To try out, adjust the number of simulation to 10 so it runs faster:
results = run_training_with_parameters(word_list, 10, parameter_sets)
report_training_performance(results)
plot_performance(results)



Starting training...
Secret Word: joust

Attempt 1:


KeyboardInterrupt: 